<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import environment as E
from buffers import ReplayBuffer
from agent import D4PG_Agent
import models
from logger import Logger
from progressbar import ProgressBar
#from get_args import get_args

import os.path
import time
import re
import sys
import importlib
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [2]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        self.num_episodes = 1
        self.max_time = 50
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [3]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000
NUM_EPISODES: 1
MAX_TIME: 50


## Load the environment
<i> 
And print a bit of information contained in the wrapper class
    
Set the training mode to FALSE while interactively learning about the code

In [4]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [ ]:
env.train = False
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
actions = np.zeros((20,4))
for i in range(10):
    actions = np.random.randn(env.agent_count, env.action_size)
    #actions[:,3] += .01
    actions = np.clip(actions, -1, 1)
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
#     if np.any(dones):
#         break
#    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [ ]:
import models
import agent
importlib.reload(models)
importlib.reload(agent)
from agent import D4PG_Agent

#### Load the AGENT!

In [ ]:
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
# print(d4pg_agent.__class__.__name__)
# print(d4pg_agent.memory)
# agent.initialize_memory(10, env)

In [ ]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [ ]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(30):
    actions = d4pg_agent.act(states)
    print("ACTIONS:", actions[1])
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [ ]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = d4pg_agent.act(states) 
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    dist, probs = d4pg_agent.critic(next_states, torch.from_numpy(actions))
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

<hr><hr>

## Test out various Agent functionality

<hr><hr>

In [24]:
import models
import agent
import buffers
importlib.reload(models)
importlib.reload(agent)
importlib.reload(buffers)
from agent import D4PG_Agent
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
env.train = True
env.reset()
d4pg_agent.initialize_memory(args.pretrain, env)

Initializing memory buffer.
Taking step...
Taking step...
Taking step...
Taking step...
Taking step...
About to store memories, current length: 0
nstep memory len: 5
Stored new memory, new length is: 1
Stored new memory, new length is: 2
Stored new memory, new length is: 3
Stored new memory, new length is: 4
Stored new memory, new length is: 5
Stored new memory, new length is: 6
Stored new memory, new length is: 7
Stored new memory, new length is: 8
Stored new memory, new length is: 9
Stored new memory, new length is: 10
Stored new memory, new length is: 11
Stored new memory, new length is: 12
Stored new memory, new length is: 13
Stored new memory, new length is: 14
Stored new memory, new length is: 15
Stored new memory, new length is: 16
Stored new memory, new length is: 17
Stored new memory, new length is: 18
Stored new memory, new length is: 19
Stored new memory, new length is: 20
Taking step...
About to store memories, current length: 20
nstep memory len: 5
Taking step...
About to 

Stored new memory, new length is: 141
Stored new memory, new length is: 142
Stored new memory, new length is: 143
Stored new memory, new length is: 144
Stored new memory, new length is: 145
Stored new memory, new length is: 146
Stored new memory, new length is: 147
Stored new memory, new length is: 148
Stored new memory, new length is: 149
Stored new memory, new length is: 150
Stored new memory, new length is: 151
Stored new memory, new length is: 152
Stored new memory, new length is: 153
Stored new memory, new length is: 154
Stored new memory, new length is: 155
Stored new memory, new length is: 156
Stored new memory, new length is: 157
Stored new memory, new length is: 158
Stored new memory, new length is: 159
Stored new memory, new length is: 160
Taking step...
About to store memories, current length: 160
nstep memory len: 5
Taking step...
About to store memories, current length: 160
nstep memory len: 5
Taking step...
About to store memories, current length: 160
nstep memory len: 5


Stored new memory, new length is: 282
Stored new memory, new length is: 283
Stored new memory, new length is: 284
Stored new memory, new length is: 285
Stored new memory, new length is: 286
Stored new memory, new length is: 287
Stored new memory, new length is: 288
Stored new memory, new length is: 289
Stored new memory, new length is: 290
Stored new memory, new length is: 291
Stored new memory, new length is: 292
Stored new memory, new length is: 293
Stored new memory, new length is: 294
Stored new memory, new length is: 295
Stored new memory, new length is: 296
Stored new memory, new length is: 297
Stored new memory, new length is: 298
Stored new memory, new length is: 299
Stored new memory, new length is: 300
Taking step...
About to store memories, current length: 300
nstep memory len: 5
Taking step...
About to store memories, current length: 300
nstep memory len: 5
Taking step...
About to store memories, current length: 300
nstep memory len: 5
Taking step...
About to store memories

Stored new memory, new length is: 423
Stored new memory, new length is: 424
Stored new memory, new length is: 425
Stored new memory, new length is: 426
Stored new memory, new length is: 427
Stored new memory, new length is: 428
Stored new memory, new length is: 429
Stored new memory, new length is: 430
Stored new memory, new length is: 431
Stored new memory, new length is: 432
Stored new memory, new length is: 433
Stored new memory, new length is: 434
Stored new memory, new length is: 435
Stored new memory, new length is: 436
Stored new memory, new length is: 437
Stored new memory, new length is: 438
Stored new memory, new length is: 439
Stored new memory, new length is: 440
Taking step...
About to store memories, current length: 440
nstep memory len: 5
Taking step...
About to store memories, current length: 440
nstep memory len: 5
Taking step...
About to store memories, current length: 440
nstep memory len: 5
Taking step...
About to store memories, current length: 440
nstep memory len

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 633
Stored new memory, new length is: 634
Stored new memory, new length is: 635
Stored new memory, new length is: 636
Stored new memory, new length is: 637
Stored new memory, new length is: 638
Stored new memory, new length is: 639
Stored new memory, new length is: 640
Taking step...
About to store memories, current length: 640
nstep memory len: 5
Taking step...
About to store memories, current length: 640
nstep memory len: 5
Taking step...
About to store memories, current length: 640
nstep memory len: 5
Taking step...
About to store memories, current length: 640
nstep memory len: 5
Taking step...
About to store memories, current length: 640
nstep memory len: 5
Stored new memory, new length is: 641
Stored new memory, new length is: 642
Stored new memory, new length is: 643
Stored new memory, new length is: 644
Stored new memory, new length is: 645
Stored new memory, new length is: 646
Stored new memory, new length is: 647
Stored new memory, new length is: 648
Stored new memory, new le

Stored new memory, new length is: 775
Stored new memory, new length is: 776
Stored new memory, new length is: 777
Stored new memory, new length is: 778
Stored new memory, new length is: 779
Stored new memory, new length is: 780
Taking step...
About to store memories, current length: 780
nstep memory len: 5
Taking step...
About to store memories, current length: 780
nstep memory len: 5
Taking step...
About to store memories, current length: 780
nstep memory len: 5
Taking step...
About to store memories, current length: 780
nstep memory len: 5
Taking step...
About to store memories, current length: 780
nstep memory len: 5
Stored new memory, new length is: 781
Stored new memory, new length is: 782
Stored new memory, new length is: 783
Stored new memory, new length is: 784
Stored new memory, new length is: 785
Stored new memory, new length is: 786
Stored new memory, new length is: 787
Stored new memory, new length is: 788
Stored new memory, new length is: 789
Stored new memory, new length 

Stored new memory, new length is: 916
Stored new memory, new length is: 917
Stored new memory, new length is: 918
Stored new memory, new length is: 919
Stored new memory, new length is: 920
Taking step...
About to store memories, current length: 920
nstep memory len: 5
Taking step...
About to store memories, current length: 920
nstep memory len: 5
Taking step...
About to store memories, current length: 920
nstep memory len: 5
Taking step...
About to store memories, current length: 920
nstep memory len: 5
Taking step...
About to store memories, current length: 920
nstep memory len: 5
Stored new memory, new length is: 921
Stored new memory, new length is: 922
Stored new memory, new length is: 923
Stored new memory, new length is: 924
Stored new memory, new length is: 925
Stored new memory, new length is: 926
Stored new memory, new length is: 927
Stored new memory, new length is: 928
Stored new memory, new length is: 929
Stored new memory, new length is: 930
Stored new memory, new length 

In [16]:
d4pg_agent.initialize_memory(args.pretrain, env)

Memory already filled, length: 1000


In [12]:
args.max_time = 100

In [13]:
#w_old = torch.zeros(d4pg_agent.actor.output.weight.data[0].shape)
for episode in range(1, 2):
    # Begin each episode with a clean environment
    env.reset()
    # Get initial state
    states = env.states
    scores = np.zeros(env.agent_count)
    # Gather experience for a maximum amount of steps, or until Done,
    # whichever comes first
    for t in range(args.max_time):
        actions = d4pg_agent.act(states)
        next_states, rewards, dones = env.step(actions)
        d4pg_agent.step(states, actions, rewards, next_states)
        states = next_states

        scores += rewards
        if np.any(dones):
            break
    print("Episode rewards: ", scores.mean())
    #print(scores)
    #print(len(d4pg_agent.memory))
    d4pg_agent.reset_nstep_memory()

Stored new memory, length is: 1001
Stored new memory, length is: 1002
Stored new memory, length is: 1003
Stored new memory, length is: 1004
Stored new memory, length is: 1005
Stored new memory, length is: 1006
Stored new memory, length is: 1007
Stored new memory, length is: 1008
Stored new memory, length is: 1009
Stored new memory, length is: 1010
Stored new memory, length is: 1011
Stored new memory, length is: 1012
Stored new memory, length is: 1013
Stored new memory, length is: 1014
Stored new memory, length is: 1015
Stored new memory, length is: 1016
Stored new memory, length is: 1017
Stored new memory, length is: 1018
Stored new memory, length is: 1019
Stored new memory, length is: 1020
Stored new memory, length is: 1021
Stored new memory, length is: 1022
Stored new memory, length is: 1023
Stored new memory, length is: 1024
Stored new memory, length is: 1025
Stored new memory, length is: 1026
Stored new memory, length is: 1027
Stored new memory, length is: 1028
Stored new memory, l

Stored new memory, length is: 1235
Stored new memory, length is: 1236
Stored new memory, length is: 1237
Stored new memory, length is: 1238
Stored new memory, length is: 1239
Stored new memory, length is: 1240
Stored new memory, length is: 1241
Stored new memory, length is: 1242
Stored new memory, length is: 1243
Stored new memory, length is: 1244
Stored new memory, length is: 1245
Stored new memory, length is: 1246
Stored new memory, length is: 1247
Stored new memory, length is: 1248
Stored new memory, length is: 1249
Stored new memory, length is: 1250
Stored new memory, length is: 1251
Stored new memory, length is: 1252
Stored new memory, length is: 1253
Stored new memory, length is: 1254
Stored new memory, length is: 1255
Stored new memory, length is: 1256
Stored new memory, length is: 1257
Stored new memory, length is: 1258
Stored new memory, length is: 1259
Stored new memory, length is: 1260
Stored new memory, length is: 1261
Stored new memory, length is: 1262
Stored new memory, l

In [ ]:
d4pg_agent.actor.fc1.weight.data[0]

## Learn about how the Categorical Bellman step works

In [ ]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [ ]:
q, probs, log_probs = critic(states, torch.from_numpy(actions))

In [ ]:
print(probs.shape)
print(log_probs.shape)
print(q)

In [ ]:
rewards

In [ ]:
class Container():
    def __init__(self):
        pass
c = Container()
    

In [ ]:
c.vmin = -10
c.vmax = 10
c.natoms = 51
c.gamma = .99
c.atoms = torch.linspace(vmin, vmax, natoms)
c.delta_z = (vmax - vmin) / (natoms -1)
c.r = torch.tensor(rewards).unsqueeze(-1)

c.probs = probs.detach()
c.q_next = (probs * atoms).sum()


In [ ]:
def print_vars(x):
    for arg in vars(x):
        print("{}: {}".format(arg.upper(), getattr(x, arg)))

In [ ]:
c.atoms

### projected atoms
<html><i>
<b>atoms.view(1,-1)</b> becomes shape [1, num_atoms]
<br>
<b>r</b> is unsqueezed in the last (-1) dimension, so it's shape [20,1]
<br>
the result is a tensor that holds an offset (projected) version of the atoms for each reward instance

In [ ]:
np.set_printoptions(suppress=False)

#### tz = projected atoms, atoms (values) projected by scaling and offsetting via the bellman equation


In [ ]:
tz = c.r + c.gamma * c.atoms.view(1,-1)
tz.clamp_(vmin, vmax)

#### computes "bj" from the pseudo-code

In [ ]:
b = (tz - c.vmin) / c.delta_z
b[0].numpy()

#### l/u in the psuedocode are LOWER and UPPER bounds on the supports

In [ ]:
l = b.floor().long()
u = b.ceil().long()
print(l[0])
print(u[0])

#### m_l/m_u are computed in the pseudocode under "distribute the probability of tz", but still a bit opaque to me on how it should be used

In [ ]:
# version from GITHUB implementations

gdml = (u.float() + (l == u).float() - b) * c.probs
gdml[0].numpy()

In [ ]:
# version from GITHUB implementations

gdmu = (b - l.float()) * c.probs
gdmu[0].numpy()

In [ ]:
# version from the PAPER

pdml = l.float() + (c.probs * (u.float()-b))
pdml[0].numpy()

In [ ]:
# version from the PAPER

pdmu = u.float() + (c.probs * (b - l.float()))
pdmu[0].numpy()

In [ ]:
target_prob.size(0)

In [ ]:
c.probs[0].numpy()

In [ ]:
target_prob = torch.tensor(np.zeros(probs.size()))
for i in range(target_prob.size(0)):
    target_prob[i].index_add_(0, l[i].long(), gdml[i].double())
    target_prob[i].index_add_(0, u[i].long(), gdmu[i].double())
target_prob[0].numpy()

In [ ]:
# zip dml and dmu together offset by 1 index and subtract 1, receive amount to add to probs!
x = zip()

In [ ]:
for i in range(c.probs.size(1)):
    t = target_prob[0].numpy()[i]
    p = c.probs[0].numpy()[i]
    pad = 8
    print(str(t)[:pad], str(p)[:pad], str(t-p)[:pad])

In [ ]:
env.close()